For this to work, the following changes need to be done to graphix: in `base_backend.py`

First, in `__init__()`, add the line `self.preparation_bank = dict()` to create this new `preparation_bank` attribute. The constructor should look like

In [ ]:
def __init__(
        self,
        state: State,
        node_index: NodeIndex | None = None,
        pr_calc: bool = True,
        rng: Generator | None = None,
    ):
        """Construct a backend.

        Parameters
        ----------
            pr_calc : bool
                whether or not to compute the probability distribution before choosing the measurement result.
                if False, measurements yield results 0/1 with 50% probabilities each.
            node_index : NodeIndex
                mapping between node numbers and qubit indices in the internal state of the backend.
            state : State
                internal state of the backend: instance of Statevec, DensityMatrix, or MBQCTensorNet.

        """
        self.__state = state
        if node_index is None:
            self.__node_index = NodeIndex()
        else:
            self.__node_index = node_index.copy()
        if not isinstance(pr_calc, bool):
            raise TypeError("`pr_calc` should be bool")
        
        self.preparation_bank = dict()

        # whether to compute the probability
        self.__pr_calc = pr_calc
        self.__rng = ensure_rng(rng)

Then: change the `add_nodes` method to the following

In [ ]:
def add_nodes(self, nodes, data=BasicStates.PLUS) -> None:
    """Add new qubit(s) to statevector in argument and assign the corresponding node number to list self.node_index.

    Parameters
    ----------
    nodes : list of node indices
    """
    # print(self.preparation_bank)
    # print(self.state)
    normal_nodes = []
    normal_data = []
    for node in nodes:
        if node in self.preparation_bank:
            self.state.add_nodes(nqubit=1, data=self.preparation_bank[node])
            self.node_index.extend([node])
        else:
            # if node != 0:
            #     print("Adding node not in bank ???")
            normal_nodes.append(node)
            idx = nodes.index(node)
            if isinstance(data, list):
                normal_data.append(data[idx])
            else:
                normal_data.append(data)

    self.state.add_nodes(nqubit=len(normal_nodes), data=normal_data)
    self.node_index.extend(normal_nodes)


Then, this works

In [1]:
import graphix.command
from graphix.random_objects import Circuit, rand_circuit
from graphix.states import BasicStates
from graphix.sim.density_matrix import DensityMatrixBackend
from graphix.sim.statevec import StatevectorBackend, Statevec
from graphix.pauli import Pauli
from graphix.fundamentals import IXYZ
import stim
from veriphix.client import Client, Secrets, CircuitUtils
import veriphix.client
import numpy as np

/Users/sabdulsa/Codes/veriphix/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:

nqubits = 2
depth = 100
circuit = rand_circuit(nqubits, depth)
pattern = circuit.transpile().pattern 
pattern.minimize_space()

print(f"Number of nodes in the pattern : {pattern.n_node}")

## Measure output nodes, to have classical output
classical_output = pattern.output_nodes
for onode in classical_output:
    pattern.add(graphix.command.M(node=onode))



secrets = Secrets(r=True, a=True, theta=True)
client = Client(pattern=pattern, secrets=secrets)

n_good_output = 0

for _i in range(1):
    backend = DensityMatrixBackend()
    client.delegate_pattern(backend=backend)

Number of nodes in the pattern : 870
